In [1]:
# import packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import csv

In [2]:
# load data: is "7_transformed_variables" the latest version until now?

data = pd.read_csv("/workspaces/bakery_sales_prediction/sourcedata/cleaned_data/7_transformed_variables-2.csv")
print(data.head(), "\n")

        Datum  Bewoelkung  Temperatur  Windgeschwindigkeit  KielerWoche  \
0  2013-07-01         6.0     17.8375                 15.0          0.0   
1  2013-07-01         6.0     17.8375                 15.0          0.0   
2  2013-07-01         6.0     17.8375                 15.0          0.0   
3  2013-07-01         6.0     17.8375                 15.0          0.0   
4  2013-07-01         6.0     17.8375                 15.0          0.0   

   Warengruppe      Umsatz  Feiertage  Ferientage  Wetterklasse  Niederschlag  \
0          1.0  148.828353        0.0         1.0             6           0.3   
1          2.0  535.856285        0.0         1.0             6           0.3   
2          3.0  201.198426        0.0         1.0             6           0.3   
3          4.0   65.890169        0.0         1.0             6           0.3   
4          5.0  317.475875        0.0         1.0             6           0.3   

   year  month  weekday  season  temp_diff_next_day  
0  2013 

In [3]:
# Define categorical features
categorical_features = ['KielerWoche', 'Warengruppe', 'Feiertage', 'Ferientage', 'Wetterklasse', 'year', 'month', 'weekday', 'season']

# Inspect data types and unique values for categorical columns
print(data[categorical_features].dtypes)
print("Unique Values:\n",data[categorical_features].apply(lambda x: x.unique()))

# Ensure categorical columns are treated as categories
for col in categorical_features:
    data[col] = data[col].astype('category')



KielerWoche     float64
Warengruppe     float64
Feiertage       float64
Ferientage      float64
Wetterklasse      int64
year              int64
month             int64
weekday           int64
season            int64
dtype: object
Unique Values:
 KielerWoche                                  [0.0, 1.0]
Warengruppe              [1.0, 2.0, 3.0, 4.0, 5.0, 6.0]
Feiertage                                    [0.0, 1.0]
Ferientage                                   [1.0, 0.0]
Wetterklasse                         [6, 2, 7, 4, 0, 8]
year               [2013, 2014, 2015, 2016, 2017, 2018]
month           [7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6]
weekday                           [0, 2, 3, 6, 4, 1, 5]
season                                     [2, 3, 4, 1]
dtype: object


In [4]:
# Encode categorical variables using pd.get_dummies
features = pd.get_dummies(data[categorical_features], drop_first=True, dtype=int)

# Include any numeric columns that are not categorical
features[['Bewoelkung', 'Temperatur', 'Windgeschwindigkeit', 'Umsatz', 'Niederschlag', 'temp_diff_next_day']] = data[['Bewoelkung', 'Temperatur', 'Windgeschwindigkeit', 'Umsatz', 'Niederschlag', 'temp_diff_next_day']]

# Construct the prepared data set including the dependent variable ('label')
prepared_data = pd.concat([data[['Umsatz']], features], axis=1).dropna()

print(prepared_data.shape)
prepared_data.head()

(6998, 45)


,Umsatz,KielerWoche_1.0,Warengruppe_2.0,Warengruppe_3.0,Warengruppe_4.0,Warengruppe_5.0,Warengruppe_6.0,Feiertage_1.0,Ferientage_1.0,Wetterklasse_2,...,weekday_6,season_2,season_3,season_4,Bewoelkung,Temperatur,Windgeschwindigkeit,Umsatz,Niederschlag,temp_diff_next_day
0,148.828353,0,0,0,0,0,0,0,1,0,...,0,1,0,0,6.0,17.8375,15.0,148.828353,0.3,-0.9250
1,535.856285,0,1,0,0,0,0,0,1,0,...,0,1,0,0,6.0,17.8375,15.0,535.856285,0.3,-0.9250
2,201.198426,0,0,1,0,0,0,0,1,0,...,0,1,0,0,6.0,17.8375,15.0,201.198426,0.3,-0.9250
3,65.890169,0,0,0,1,0,0,0,1,0,...,0,1,0,0,6.0,17.8375,15.0,65.890169,0.3,-0.9250
4,317.475875,0,0,0,0,1,0,0,1,0,...,0,1,0,0,6.0,17.8375,15.0,317.475875,0.3,2.3125


In [5]:
# Handle missing values by removing rows with any missing values
prepared_data = prepared_data.dropna()

# Display the shape of the prepared data set
print(prepared_data.shape)
# Display the first few rows of the prepared data set
prepared_data.head()

(6998, 45)


,Umsatz,KielerWoche_1.0,Warengruppe_2.0,Warengruppe_3.0,Warengruppe_4.0,Warengruppe_5.0,Warengruppe_6.0,Feiertage_1.0,Ferientage_1.0,Wetterklasse_2,...,weekday_6,season_2,season_3,season_4,Bewoelkung,Temperatur,Windgeschwindigkeit,Umsatz,Niederschlag,temp_diff_next_day
0,148.828353,0,0,0,0,0,0,0,1,0,...,0,1,0,0,6.0,17.8375,15.0,148.828353,0.3,-0.9250
1,535.856285,0,1,0,0,0,0,0,1,0,...,0,1,0,0,6.0,17.8375,15.0,535.856285,0.3,-0.9250
2,201.198426,0,0,1,0,0,0,0,1,0,...,0,1,0,0,6.0,17.8375,15.0,201.198426,0.3,-0.9250
3,65.890169,0,0,0,1,0,0,0,1,0,...,0,1,0,0,6.0,17.8375,15.0,65.890169,0.3,-0.9250
4,317.475875,0,0,0,0,1,0,0,1,0,...,0,1,0,0,6.0,17.8375,15.0,317.475875,0.3,2.3125


In [6]:
# Save the updated data to new CSV files
prepared_data.to_csv('../sourcedata/cleaned_data/8_encoded_variables-2.csv', sep=',', index=False)

# and save to pickles!
prepared_data.to_pickle('../sourcedata/cleaned_data/8_encoded_variables-2.pkl')